In [10]:
import radiomics as rad
import nibabel as nib
import SimpleITK as sitk
import numpy as np

In [6]:
im = nib.load("/home/cerebriu/data/datasets/processed_final/cmb_dou/Data/DOU-01-CMB-1/Annotations/DOU-01-CMB-1.nii.gz")
mri = nib.load("/home/cerebriu/data/datasets/processed_final/cmb_dou/Data/DOU-01-CMB-1/MRIs/DOU-01-CMB-1.nii.gz")

In [7]:
im_Data = im.get_fdata()
mri_Data = mri.get_fdata()

In [15]:
from radiomics import featureextractor
import SimpleITK as sitk

# Convert Numpy arrays to SimpleITK images (if not loading from files)
image_sitk = sitk.GetImageFromArray(mri_Data)
mask_sitk = sitk.GetImageFromArray(im_Data.astype(np.uint8))

# Set up the PyRadiomics feature extractor
params = {}
params['label'] = 1  # Label of interest
params['calculateAllShapeFeatures'] = True  # Calculating all shape features

extractor = rad.featureextractor.RadiomicsFeatureExtractor(**params)
extractor.enableFeatureClassByName('shape')  # Enable shape features only for speed

# Extract features
result = extractor.execute(image_sitk, mask_sitk)

# Access the sphericity
sphericity = result.get('original_shape_Sphericity')
print('Sphericity:', sphericity)


In [18]:
def calculate_sphericity(mri_data, mask_data):
    """
    Calculate the sphericity of an object in a binary mask using PyRadiomics.

    Args:
        mri_data (numpy.ndarray): The MRI data array (can be a dummy array if not used).
        mask_data (numpy.ndarray): The binary mask array where the object is labeled with 1.

    Returns:
        float or None: The sphericity of the object, or None if sphericity cannot be computed.
    """
    # Convert numpy arrays to SimpleITK images
    image_sitk = sitk.GetImageFromArray(mri_data)
    mask_sitk = sitk.GetImageFromArray(mask_data.astype(np.uint8))  # Ensure mask is in correct data type

    # Set up the PyRadiomics feature extractor with specific parameters
    params = {
        'label': 1,  # Assuming the object of interest is labeled as 1 in the mask
        'calculateAllShapeFeatures': True
    }
    extractor = featureextractor.RadiomicsFeatureExtractor(**params)
    extractor.enableFeatureClassByName('shape')  # Enable only shape features to reduce computation time

    # Extract features
    result = extractor.execute(image_sitk, mask_sitk)

    # Retrieve and return the sphericity
    sphericity = result.get('original_shape_Sphericity')
    if sphericity is not None:
        return float(sphericity)
    else:
        return None

In [19]:
calculate_sphericity(mri_Data, im_Data)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


0.8049652603910995

In [14]:
print((sphericity.shape))


()


In [34]:
def calculate_radiomics_features(mri_data, mask_data):
    """
    Calculate a comprehensive set of radiomics features for an object in a binary mask using PyRadiomics.

    Args:
        mri_data (numpy.ndarray): The MRI data array.
        mask_data (numpy.ndarray): The binary mask array where the object is labeled with 1.

    Returns:
        dict: A dictionary containing various radiomics features.
    """
    # Convert numpy arrays to SimpleITK images
    image_sitk = sitk.GetImageFromArray(mri_data)
    mask_sitk = sitk.GetImageFromArray(mask_data.astype(np.uint8))  # Ensure mask is in correct data type

    unique_labels = np.unique(mask_data)
    assert len(unique_labels) == 2 and 0 in unique_labels and 1 in unique_labels, 'Mask must be binary'

    # Set up the PyRadiomics feature extractor with specific parameters
    settings = {
        'binWidth': 25,
        'resampledPixelSpacing': None,
        'interpolator': sitk.sitkBSpline,
        'enableCExtensions': True
    }

    # Set up the parameters for PyRadiomics to extract all features
    params = {
        'label': 1,  # Assuming the object of interest is labeled as 1 in the mask
        'calculateAllShapeFeatures': True,
        'calculateAllFirstOrderFeatures': True,
        'calculateAllGLCMFeatures': True,
        'calculateAllGLRLMFeatures': True,
        'calculateAllGLSZMFeatures': True,
        'calculateAllGLDMFeatures': True,
        'calculateAllNGTDMFeatures': True
    }

    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    extractor.enableAllFeatures()  # You can specify certain features classes if needed

    # Extract features
    result = extractor.execute(image_sitk, mask_sitk)

    # Convert the result to a clean dictionary
    features_dict = {key: result[key] for key in result.keys()}

    return features_dict

In [35]:
radiomics_features = calculate_features(mri_Data, im_Data, 1)

AttributeError: 'RadiomicsFeatureExtractor' object has no attribute 'updateParams'

In [ ]:
# Print results
print("Extracted Radiomics Features:")
for feature, value in radiomics_features.items():
    print(f"{feature}: {value}")
